In [1]:
import urllib.request as req
import json
import pandas as pd
import numpy as np
import requests
from sqlalchemy import create_engine

In [2]:
website = pd.DataFrame(np.array([['01', 'hahow', 'https://hahow.in/courses'],
                                 ['02', 'tibame', 'https://www.tibame.com/courselibrary'],
                                 ['03', 'pressplay', 'https://www.pressplay.cc/'],
                                 ['04', 'yotta', 'https://www.yottau.com.tw/home']]),
                                 columns = ['website_id', 'website_name', 'website_url'])
category = pd.DataFrame(np.array([['01', '語言'],
                                  ['02', '藝術'],
                                  ['03', '設計'],
                                  ['04', '多媒體設計'],
                                  ['05', '程式'],
                                  ['06', '行銷'],
                                  ['07', '投資理財'],
                                  ['08', '職場技能'],
                                  ['09', '生活品味']]),
                                  columns = ['category_id', 'category_name'])
status = pd.DataFrame(np.array([['01', 'SUCCESS'],
                                ['02', 'PUBLISHED'],
                                ['03', 'INCUBATING']]),
                                columns = ['status_id', 'status_description'])
course = pd.DataFrame(columns = ['course_id', 'course_name', 'website_id', 'course_intro', 'course_price', 'course_time', 'teacher_id', 'category_id', 'course_url', 'course_img_url', 'status_id', 'course_tag'])
teacher = pd.DataFrame(columns = ['teacher_id', 'teacher_name', 'teacher_intro', 'teacher_img_url'])
review = pd.DataFrame(columns = ['review_id', 'review_star', 'review_content', 'review_time', 'course_id'])
#discount = pd.DataFrame(columns = ['c_id', 'd_id', 'd_content'])
fundraise = pd.DataFrame(columns = ['fundraise_id', 'fundraise_due_date', 'fundraise_percent', 'course_id'])

In [3]:
course_count = 0
teacher_count = 0
fundraise_count = 0
review_count = 0
discount_count = 0

# Hahow

## Web_crawler

In [4]:
for i in range(29):
    url = "https://api.hahow.in/api/courses?limit=24&page=" + str(i)
    request = req.Request(url, headers = {
        "Content-Type":"application/json; charset=utf-8",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    })
    with req.urlopen(request) as response:
        result = response.read().decode("utf-8")
    result = json.loads(result)

    for j in range(len(result['data'])):
        #course
        data_course = {'course_id':str(course_count + 1).zfill(5),
                       'course_name':result['data'][j]['title'],
                       'website_id':'01',
                       'course_intro':result['data'][j]['metaDescription'],
                       'course_price':result['data'][j]['price'],
                       'teacher_id':str(course_count + 1).zfill(5),
                       'course_url':"https://api.hahow.in/api/courses/" + result['data'][j]['_id'],
                       'course_img_url':result['data'][j]['coverImage']['url']}
        course = course.append(data_course, ignore_index = True)
        #teahcer
        if result['data'][j]['owner']['name'] not in teacher['teacher_name']:
            data_teacher = {'teacher_id':str(teacher_count + 1).zfill(5),
                            'teacher_name':result['data'][j]['owner']['name']}
            teacher = teacher.append(data_teacher, ignore_index = True)
            course['teacher_id'] = str(teacher_count + 1).zfill(5)
            teacher_count += 1
        else:
            course['teacher_id'] = teacher['teacher_id'][teacher['teacher_name'].index(result['data'][j]['owner']['name'])]
        #fundraise
        try:
            data_fundraise = {'fundraise_id':str(fundraise_count + 1).zfill(5),
                              'fundraise_due_date':result['data'][j]['campaign']['expireDate'],
                              'course_id':str(course_count + 1).zfill(5)}
            fundraise = fundraise.append(data_fundraise, ignore_index = True)
            fundraise_count += 1
        except KeyError:
            pass
        #discount
        #data_discount = {'c_id':str(course_count + 1).zfill(5)
        #                , 'd_id':str(discount_count + 1).zfill(5)
        #                , 'd_content':'preOrderedPrice:' + str(result['data'][j]['basePricingInfo']['preOrderedPrice'])}
        #discount = discount.append(data_discounts, ignore_index = True)
        #discount_count += 1
        course_count += 1

In [5]:
for i in range(len(course)):
    url = course['course_url'][i]
    request = req.Request(url, headers = {
            "Content-Type":"application/json; charset=utf-8",
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
        })
    with req.urlopen(request) as response:
        result = response.read().decode("utf-8")
    result = json.loads(result)

    course['course_time'][i] = result['estimatedCourseVideoLengthInMins']

    try:
        if result['group']['title'] == '語言':
            course['category_id'][i] = '01'
        elif result['group']['title'] == '藝術':
            course['category_id'][i] = '02'
        elif result['group']['title'] == '設計':
            course['category_id'][i] = '03'
        elif result['group']['title'] == '程式':
            course['category_id'][i] = '05'
        elif result['group']['title'] == '行銷':
            course['category_id'][i] = '06'
        elif result['group']['title'] == '投資理財':
            course['category_id'][i] = '07'
        elif result['group']['title'] == '職場技能':
            course['category_id'][i] = '08'
        elif result['group']['title'] == '音樂' or result['group']['title'] == '攝影' or result['group']['title'] == '人文' or result['group']['title'] == '手作' or result['group']['title'] == '生活品味':
            course['category_id'][i] = '09'
    except KeyError:
        pass
    
    if result['status'] == 'SUCCESS':
        course['status_id'][i] = '01'
    elif result['status'] == 'PUBLISHED':
        course['status_id'][i] = '02'
    elif result['status'] == 'INCUBATING':
        course['status_id'][i] = '03'

    try:
        teacher['teacher_intro'][i] = result['owner']['metaDescription']
    except KeyError:
        pass
    
    try:
        teacher['teacher_img_url'][i] = result['owner']['profileImageUrl']
    except KeyError:
        pass

In [6]:
for i in range(len(course)):
    url = course['course_url'][i] + "/comments?limit=20&page=0"
    request = req.Request(url, headers = {
            "Content-Type":"application/json; charset=utf-8",
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
        })
    with req.urlopen(request) as response:
        result = response.read().decode("utf-8")
    result = json.loads(result)
    
    for j in range(len(result)):
        data_review = {'review_id':str(review_count + 1).zfill(5),
                       'review_content':result[j]['content'],
                       'review_time':result[j]['editTime'],
                       'course_id':course['course_id'][i]}
        review = review.append(data_review, ignore_index = True)
        review_count += 1 

# TibaMe

In [7]:
url = "https://api.tibame.com/courselib/user/courseList"

requestData = {
    "orderBy": "popular",
    "sortType": "DESC",
    "isPublic": 1,
    "numPerPage": 400,
    "isCategory": False,
    "isSubCategory": False
}

headers = {
    "Content-Type":"application/json; charset=utf-8",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
}

request = req.Request(url, headers = headers, data = json.dumps(requestData).encode("utf-8"))

In [8]:
with req.urlopen(request) as response:
    result = response.read().decode("utf-8")
#print(result)

result = json.loads(result)

## Getting Data

In [9]:
for i in range(342):
    url2 = " https://api-c2c.tibame.com/v1/c2c/user-courses/course/" + str(result['data']['results'][i]['uid']) + "/detail"
    headers2 = {
        "Content-Type":"application/json; charset=utf-8",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }
    request2 = req.Request(url2, headers = headers2)
    
    with req.urlopen(request2) as response2:
        result2 = response2.read().decode("utf-8")

    result2 = json.loads(result2)
    # print(result2['data']['data']) 

    c_data = {
        'course_id': str(course_count + 1).zfill(5),
        'course_name': result['data']['results'][i]['courseName'],
        'website_id': '02',
        'course_intro': result['data']['results'][i]['brief'],
        'course_price': result['data']['results'][i]['price'],
        'course_time': result2['data']['data']['videoTime'], #sec
        'teacher_id': str(course_count + 1).zfill(5),
        #'c_start_time'
        #'c_buyers': result['data']['results'][i]['userCount'],
        #'cg_id': result['data']['results'][i]['categoryUid'],
        'course_url': 'https://www.tibame.com/course/' + str(result['data']['results'][i]['uid']),
        'course_img_url': result['data']['results'][i]['coverPic'],
        'status_id': '01'
        #'course_tag'
    }
    course = course.append(c_data, ignore_index = True)
    
    if result['data']['results'][i]['teacher']['nickname'] not in teacher['teacher_name']:
        t_data = {
            'teacher_id': str(teacher_count + 1).zfill(5),
            'teacher_name': result['data']['results'][i]['teacher']['nickname'],
            'teacher_intro': result2['data']['data']['teacher']['introContent']
            #'teacher_img_url'
        }
        teacher = teacher.append(t_data, ignore_index = True)
        course['teacher_id'] = str(teacher_count + 1).zfill(5)
        teacher_count += 1
    else:
        course['teacher_id'] = teacher['teacher_id'][teacher['teacher_name'].index(result['data']['results'][i]['teacher']['nickname'])]
    '''w_data = {
        # 'w_id': result['data']['results'][i]['uid'],
        # 'w_name'
        # 'w_url'
    }
    website = website.append(w_data, ignore_index = True)
    '''
    
    '''
    if result['data']['results'][i]['discountPrice'] != 0:
        d_data = {
            'd_id': str(discount_count + 1).zfill(5),
            'c_id': str(course_count + 1).zfill(5),
            'd_content': result['data']['results'][i]['discountPrice']
        }
        discounts = discounts.append(d_data, ignore_index = True)
        discount_count += 1
    '''
    
    '''
    r_data = {
        # 'r_id'
        # 'r_stars' 
        # 'r_content'
        # 'c_id'
    }
    #review = review.append(r_data, ignore_index = True)
    '''
    
    '''
    f_data = {
        # 'f_id'
        # 'f_due_date'
        # 'f_process'
        #'c_id'
    }
    #fundraise = fundraise.append(f_data, ignore_index = True)
    '''
    
    course_count += 1

# Result

In [11]:
course

,course_id,course_name,website_id,course_intro,course_price,course_time,teacher_id,category_id,course_url,course_img_url,status_id,course_tag
0,00001,小資族圓夢計畫，第一次買房收租就上手,01,這門課程將教會你各種房產投資的知識，包括整層收租、隔套收租、二房東等，讓小資族的你可以了解不...,3280,120,01031,07,https://api.hahow.in/api/courses/6136ed9fafdea...,https://images.api.hahow.in/images/6166a3dc2ba...,03,NaN
1,00002,J Team 小學堂－英雄聯盟爬分 10 大重點,01,想提升《英雄聯盟》牌位嗎？由 PLANET9 與電競戰隊 J Team 合作，帶來職業等級的...,990,100,01031,09,https://api.hahow.in/api/courses/6130753a26d20...,https://images.api.hahow.in/images/614c237ddeb...,03,NaN
2,00003,Knitting好好玩！自己織圍巾、披肩、羊毛帽,01,輕鬆有趣的編織課，打造獨一無二的風格穿搭。不論是基本針法還是觀念小撇步，都是編織前你想知道的...,2480,150,01031,09,https://api.hahow.in/api/courses/6107cd6c3cc7a...,https://images.api.hahow.in/images/6149d8fc57d...,03,NaN
3,00004,放下酒譜！跟著 Mars 飛向浩瀚無垠調酒宇宙,01,To Infinity｜\n是用現代科學技術與無限的想像力來重新詮釋每一杯經典調酒，不再只是...,3980,360,01031,09,https://api.hahow.in/api/courses/61666a458fc5c...,https://images.api.hahow.in/images/617824e6b2d...,03,NaN
4,00005,籌碼入門手冊：數據拆解主力思維,01,或許我們都認為小資族與大戶資源懸殊，怎麼可能追上他們呢？\n那你就錯了！其實你擁有的比想像多...,3600,260,01031,07,https://api.hahow.in/api/courses/611dbe96be4b2...,https://images.api.hahow.in/images/614ad1c2deb...,03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1026,01027,【聽課】第二屆百業千師計劃—未來的工作模式：遠距工作與非典型僱用,02,我們可以從趨勢面聊聊未來可能的勞動模式，也會分享在疫情時代下有什麼樣的企業人才部署戰略，從雙...,0,323,01031,NaN,https://www.tibame.com/course/1151,https://cdn-static.tibame.com/course/1151/cove...,01,NaN
1027,01028,【聽課+實況】第二屆百業千師計劃—大遠端時代的溝通、協作與管理,02,講者國際工作經理豐富，且創辦 Meet.jobs 後有來自 18 國家的企業和求職用戶。將透...,0,4054,01031,NaN,https://www.tibame.com/course/1046,https://cdn-static.tibame.com/course/1046/cove...,01,NaN
1028,01029,Windows Container入門與實作,02,想知道Windows Container容器技術能為 .net 開發團隊解決什麼問題嗎？ 容...,0,4634,01031,NaN,https://www.tibame.com/course/79,https://cdn-static.tibame.com/course/79/coverP...,01,NaN
1029,01030,人資長曾華英親身教你履歷面試技巧,02,讓你了解主管的心 ！ 寫出讓人難以放手的履歷 ！,0,6463,01031,NaN,https://www.tibame.com/course/54,https://cdn-static.tibame.com/course/54/coverP...,01,NaN


In [10]:
review

,review_id,review_star,review_content,review_time,course_id
0,00001,NaN,請教一下會指導吃兵訣竅嗎？多謝,2021-11-03T09:32:04.171Z,00002
1,00002,NaN,竟然有英雄聯盟課程也太酷了吧~,2021-11-02T04:11:16.135Z,00002
2,00003,NaN,"不只是伊隆·馬斯克的SpaceX, 我已經準備好要跟火星人Mars 飛向調酒宇宙了!!!!",2021-11-02T12:45:05.923Z,00004
3,00004,NaN,想請問是上課又飲酒嗎？ 謝謝,2021-10-31T14:56:42.040Z,00004
4,00005,NaN,近期在學習如何抓個股的主力分點，並且透過主力分點進出場，但還是不太會分析，請問這部分課程可以...,2021-11-03T04:49:06.012Z,00005
...,...,...,...,...,...
11341,11342,NaN,請問購買後，課程是線上看，還是提供載點?\n課程結束可以重複看嗎?,2015-03-11T06:44:04.674Z,00688
11342,11343,NaN,所以如果連photoshop跟 Illustrator都不會的人是否比較不建議上這門課呢~?,2015-03-05T09:23:05.843Z,00688
11343,11344,NaN,需要電腦操作嗎？,2021-05-26T02:15:02.406Z,00689
11344,11345,NaN,請問，鋼筆工具，拉最後一個點時，要同時按甚麼鍵(windows)?,2017-11-07T06:13:20.299Z,00689


In [11]:
teacher

,teacher_id,teacher_name,teacher_intro,teacher_img_url
0,00001,RICHARK,是一個「全方位教練系統 Total Coaching System」，為創辦人「吳建賢Cos...,https://images.api.hahow.in/images/5d2fdd5bfe9...
1,00002,PLANET9,PLANET9提供全球玩家全方位的遊戲體驗，從「組隊」、「找教練」到「賽事舉辦」等多元化服務...,https://images.api.hahow.in/images/613071b126d...
2,00003,May｜就是愛編織,May，專職玩毛線十多年，有一間靠近海的手染編織工作室，有植物，好咖啡，和機靈的狗朋友Oreo,https://images.api.hahow.in/images/60ee725d6f7...
3,00004,Mars x 三十創意,【Mars】 為「2021年World Class世界頂尖調酒大賽」台灣冠軍。Mars自學習...,https://images.api.hahow.in/images/6179f97d7bb...
4,00005,豹投資,豹投資，專業投資人的邏輯整合大數據分析，成為多方投資策略的專業選股系統網站。\n不管在財務面...,https://images.api.hahow.in/images/6024082f030...
...,...,...,...,...
1026,01027,TibaMe,TibaMe是緯創旗下教育品牌一緯育所創的新一代學習平台，是全台最大的數位跨域人才培育加器。...,NaN
1027,01028,TibaMe,TibaMe是緯創旗下教育品牌一緯育所創的新一代學習平台，是全台最大的數位跨域人才培育加器。...,NaN
1028,01029,TibaMe,TibaMe是緯創旗下教育品牌一緯育所創的新一代學習平台，是全台最大的數位跨域人才培育加器。...,NaN
1029,01030,TibaMe,TibaMe是緯創旗下教育品牌一緯育所創的新一代學習平台，是全台最大的數位跨域人才培育加器。...,NaN


In [12]:
fundraise

,fundraise_id,fundraise_due_date,fundraise_percent,course_id
0,00001,2021-12-02T16:00:00.000Z,NaN,00001
1,00002,2021-11-01T16:00:00.000Z,NaN,00002
2,00003,2021-11-26T16:00:00.000Z,NaN,00003
3,00004,2021-11-26T16:00:00.000Z,NaN,00004
4,00005,2021-11-25T16:00:00.000Z,NaN,00005
...,...,...,...,...
208,00209,2018-07-02T16:00:00.000Z,NaN,00474
209,00210,2018-06-11T16:00:00.000Z,NaN,00480
210,00211,2018-08-22T16:00:00.000Z,NaN,00485
211,00212,2018-06-04T16:00:00.000Z,NaN,00487


In [20]:
website = {'website_name': ['Hahow', 'TibaMe', 'PressPlay', 'Yotta'],
           'website_url': ['https://hahow.in/apselect-event?id=12069&id2=d6d82961cf6f04a4de5ec39b3cbaadbf&mts_c=19ap&mts_m=ao&mts_s=ap&utm_medium=cpa&utm_source=affiliates&utm_term=d6d82961cf6f04a4de5ec39b3cbaadbf&vtm_channel=affiliatescomtw&vtm_stat_id=d6d82961cf6f04a4de5ec39b3cbaadbf&vtmz=true&rc=2',
                           'https://www.tibame.com/?utm_source=ichannels&utm_medium=affiliate_marketing&utm_campaign=all_product&gid=1f62e62097772e9f24ef509071a96dd8',
                           'https://www.pressplay.cc/', 
                           'https://www.yottau.com.tw/home'],
           'website_id': ['0001', '0002', '0003', '0004']}
website = pd.DataFrame(website)

# Insert into Database

In [22]:
# fill in your db account info
username = "postgres"
password = "judy891215"

# let's insert data
engine = create_engine("postgresql://" + username + ":" + password + "@127.0.0.1:5432/DBP")
website.to_sql('website', engine, schema = 'COURSE', if_exists = "append", index = False)
category.to_sql('category', engine, schema = 'COURSE', if_exists = "append", index = False)
status.to_sql('status', engine, schema = 'COURSE', if_exists = "append", index = False)
course.to_sql('course', engine, schema = 'COURSE', if_exists = "append", index = False)
teacher.to_sql('teacher', engine, schema = 'COURSE', if_exists = "append", index = False)
review.to_sql('review', engine, schema = 'COURSE', if_exists = "append", index = False)
fundraise.to_sql('fundraise', engine, schema = 'COURSE', if_exists = "append", index = False)